# Disease Prediction

## Importing libraries

In [2]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Data Preprocessing

### Training Image preprocessing

In [29]:
training_set = tf.keras.utils.image_dataset_from_directory(
    'Dataset/training',      # Path to the training dataset
    labels="inferred",        # Automatically infer labels from sub-directory names
    label_mode="categorical", # One-hot encode labels
    class_names=None,         # Automatically use folder names as class labels
    color_mode="rgb",         # RGB images
    batch_size=32,            # Batch size for training
    image_size=(128, 128),    # Resize all images to 128x128
    shuffle=True,             # Shuffle the dataset
    seed=42,                  # Seed for reproducibility
    validation_split=None,    # No validation split here, you already have a separate validation set
    interpolation="bilinear", # Interpolation method for resizing
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 37607 files belonging to 10 classes.


### Validation Image Preprocessing

In [4]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    'Dataset/validation',     # Path to the validation dataset
    labels="inferred",        # Automatically infer labels from sub-directory names
    label_mode="categorical", # One-hot encode labels
    class_names=None,         # Automatically use folder names as class labels
    color_mode="rgb",         # RGB images
    batch_size=32,            # Batch size for validation
    image_size=(128, 128),    # Resize all images to 128x128
    shuffle=False,            # No shuffling for validation data
    seed=42,                  # Seed for reproducibility
    validation_split=None,    # No need for validation split
    interpolation="bilinear", # Interpolation method for resizing
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 16132 files belonging to 10 classes.


## Building Model

In [5]:
# Initialize a Sequential model to build the CNN layer by layer
cnn = tf.keras.models.Sequential()

### Building Convolution Layer

In [6]:
# Initialize a Sequential model to build the CNN layer by layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=[128, 128, 3])) # 32 filters of 3x3, keeps same dimensions with padding, uses ReLU activation
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')) # Second conv layer in the block, captures deeper features
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) # Pooling layer with 2x2 window, reduces spatial dimensions by half

C:\Users\green\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Second Convolutional Block
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')) # 64 filters of 3x3, retains original dimensions with padding
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu')) # Another convolutional layer, refining features further
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) # Reduces spatial size by half again

In [8]:
# Third Convolutional Block
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')) # 128 filters for more complex feature extraction
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu')) # Second conv layer in the block, enhancing feature detection
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) # Reduces spatial size to capture only essential features

In [9]:
# Fourth Convolutional Block 
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')) # 256 filters for capturing finer details
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu')) # Additional layer for further detail extraction
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) # Again halves the dimensions

In [10]:
# Fifth Convolutional Block 
cnn.add(tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')) # 512 filters to focus on complex patterns in deep layers
cnn.add(tf.keras.layers.Conv2D(filters=512, kernel_size=3, activation='relu')) # Further refining the feature set
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) # Reduces dimensions, bringing data down to a lower spatial resolution

In [11]:
# Dropout Layer
cnn.add(tf.keras.layers.Dropout(0.25)) # Adds dropout with 25% probability to prevent overfitting in the CNN layers

In [12]:
# Flatten Layer
cnn.add(tf.keras.layers.Flatten()) # Flattens the feature map into a 1D vector for input to the Dense layer

In [13]:
# Fully Connected Dense Layer
cnn.add(tf.keras.layers.Dense(units=1500,activation='relu')) # Dense layer with 1500 neurons, learns complex relationships in the data

In [14]:
# Dropout Layer
cnn.add(tf.keras.layers.Dropout(0.4)) # Dropout with 40% probability to reduce overfitting further in dense layers

In [15]:
# Output Layer
cnn.add(tf.keras.layers.Dense(units=10, activation='softmax')) # Output layer with 10 neurons (one per class), softmax for probability distribution across classes

### Compiling and Training Phase

In [16]:
# Compile the Model
cnn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Adam optimizer with a learning rate of 0.0001, efficient for convergence
    loss='categorical_crossentropy',  # Loss function for multi-class classification
    metrics=['accuracy']  # Accuracy Metric to evaluate model performance during training and validation
)


In [17]:
cnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 126, 126, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 63, 63, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 61, 61, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 30, 30, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 28, 28, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 14, 14, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 12, 12, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 6, 6, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1500)                │       3,073,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1500)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │          15,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,800,734 (29.76 MB)

 Trainable params: 7,800,734 (29.76 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
training_history = cnn.fit(
    x=training_set,              # Training dataset
    validation_data=validation_set,  # Validation dataset
    epochs=40                    # Train for 40 epochs
)

Epoch 1/40
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 597s 505ms/step - accuracy: 0.7569 - loss: 0.9321 - val_accuracy: 0.7740 - val_loss: 0.7163
Epoch 2/40
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 589s 501ms/step - accuracy: 0.7822 - loss: 0.6874 - val_accuracy: 0.8060 - val_loss: 0.5708
Epoch 3/40
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 589s 501ms/step - accuracy: 0.8163 - loss: 0.5592 - val_accuracy: 0.8206 - val_loss: 0.5292
Epoch 4/40
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 612s 521ms/step - accuracy: 0.8367 - loss: 0.4881 - val_accuracy: 0.8388 - val_loss: 0.4803
Epoch 5/40
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 783s 666ms/step - accuracy: 0.8596 - loss: 0.4119 - val_accuracy: 0.8457 - val_loss: 0.4661
Epoch 6/40
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 797s 678ms/step - accuracy: 0.8765 - loss: 0.3521 - val_accuracy: 0.8587 - val_loss: 0.4125
Epoch 7/40
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 566s 481ms/step - accuracy: 0.8959 - loss: 0.2979 - val_accuracy: 0.8644 - val_loss: 0.4206
Epoch 8/40
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 555s 472ms/step - ac

## Evaluating Model

In [19]:
#Training set Accuracy
train_loss, train_acc = cnn.evaluate(training_set)
print('Training accuracy:', train_acc)

1176/1176 ━━━━━━━━━━━━━━━━━━━━ 125s 106ms/step - accuracy: 0.9799 - loss: 0.0715
Training accuracy: 0.980828046798706


In [20]:
#Validation set Accuracy
val_loss, val_acc = cnn.evaluate(validation_set)
print('Validation accuracy:', val_acc)

505/505 ━━━━━━━━━━━━━━━━━━━━ 54s 106ms/step - accuracy: 0.7792 - loss: 1.3006
Validation accuracy: 0.8602777123451233


In [21]:
import numpy as np
from sklearn.metrics import (balanced_accuracy_score, roc_auc_score, 
                             recall_score, precision_score, f1_score, 
                             confusion_matrix, average_precision_score)

# Step 1: Predict on the validation set
y_pred_classes = []
y_true = []

for x, y in validation_set:  # Validation set contains both images and true labels
    preds = cnn.predict(x)
    y_pred_classes.append(preds)  # Store predicted probabilities
    y_true.append(y.numpy())      # Store true labels (one-hot encoded)

# Convert list of batches into arrays
y_pred_classes = np.concatenate(y_pred_classes, axis=0)  # Predicted probabilities for all data
y_true = np.concatenate(y_true, axis=0)  # True labels for all data

# Convert one-hot encoded labels to class indices
y_true_classes = np.argmax(y_true, axis=1)
y_pred_labels = np.argmax(y_pred_classes, axis=1)

# Step 3: Calculate Metrics

# 1. Balanced Accuracy
balanced_acc = balanced_accuracy_score(y_true_classes, y_pred_labels)
print("Balanced Accuracy:", balanced_acc)

# 2. Mean AUC-ROC (One-vs-Rest for multi-class classification)
mean_auc = roc_auc_score(y_true, y_pred_classes, multi_class='ovr')
print("Mean AUC-ROC:", mean_auc)

# 3. Specificity and Sensitivity (Recall)
cm = confusion_matrix(y_true_classes, y_pred_labels)

# True Negatives (TN), False Positives (FP), False Negatives (FN), True Positives (TP)
tn = cm.sum(axis=1) - cm.diagonal()
fp = cm.sum(axis=0) - cm.diagonal()
fn = cm.sum(axis=1) - cm.diagonal()
tp = cm.diagonal()

# Specificity (TN / (TN + FP))
specificity = tn / (tn + fp)
mean_specificity = np.mean(specificity)
print("Specificity per class:", specificity)
print("Mean Specificity:", mean_specificity)

# Sensitivity (Recall: TP / (TP + FN))
sensitivity = recall_score(y_true_classes, y_pred_labels, average=None)
mean_sensitivity = recall_score(y_true_classes, y_pred_labels, average='macro')
print("Sensitivity per class:", sensitivity)
print("Mean Sensitivity:", mean_sensitivity)

# 4. F1 Score
f1 = f1_score(y_true_classes, y_pred_labels, average=None)
mean_f1 = f1_score(y_true_classes, y_pred_labels, average='macro')
print("F1 Score per class:", f1)
print("Mean F1 Score:", mean_f1)

# 5. Average Precision Score
avg_precision = average_precision_score(y_true, y_pred_classes, average=None)
mean_avg_precision = average_precision_score(y_true, y_pred_classes, average='macro')
print("Average Precision per class:", avg_precision)
print("Mean Average Precision:", mean_avg_precision)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

### Saving Model

In [22]:
cnn.save('cnn40trained_disease_model.keras')

In [23]:
training_history.history #Return Dictionary of history

{'accuracy': [0.7659212350845337,
  0.7963411211967468,
  0.8245007395744324,
  0.8460924625396729,
  0.8664344549179077,
  0.880314826965332,
  0.9008163213729858,
  0.915228545665741,
  0.9336825609207153,
  0.946047306060791,
  0.9560188055038452,
  0.9604328870773315,
  0.9680644273757935,
  0.9717339873313904,
  0.9743664860725403,
  0.9768660068511963,
  0.9765735268592834,
  0.9790198802947998,
  0.9810939431190491,
  0.9819714426994324,
  0.9825298190116882,
  0.984763503074646,
  0.9853484630584717,
  0.9839657545089722,
  0.9850825667381287,
  0.9859866499900818,
  0.9874491691589355,
  0.9867045879364014,
  0.9871566295623779,
  0.9874491691589355,
  0.9875820875167847,
  0.9875820875167847,
  0.9884064197540283,
  0.9896029829978943,
  0.9890977740287781,
  0.9893105030059814,
  0.9884064197540283,
  0.9905602931976318,
  0.9893636703491211,
  0.9899221062660217],
 'loss': [0.8331549167633057,
  0.6411571502685547,
  0.5290229320526123,
  0.4566470980644226,
  0.38879495859

In [24]:
#Recording History in json
import json
with open('training_hist.json','w') as f:
  json.dump(training_history.history,f)

In [25]:
print(training_history.history.keys())

dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])


## Accuracy Visualization

In [ ]:
epochs = [i for i in range(1,11)]
plt.plot(epochs,training_history.history['accuracy'],color='red',label='Training Accuracy')
plt.plot(epochs,training_history.history['val_accuracy'],color='blue',label='Validation Accuracy')
plt.xlabel('No. of Epochs')
plt.title('Visualization of Accuracy Result')
plt.legend()
plt.show()

In [28]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import img_to_array, load_img
import pandas as pd

# Define the directory containing images and the image size
test_dir = 'Dataset/Testing'
image_size = (128, 128)

# Prepare lists to store data
image_paths = []
images = []

# Load images and paths manually
for img_name in os.listdir(test_dir):
    if img_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
        img_path = os.path.join(test_dir, img_name)
        image_paths.append(img_name)
        img = load_img(img_path, target_size=image_size)
        img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
        images.append(img_array)

# Convert list of images to a numpy array
images = np.array(images)

# Run predictions on the loaded images
predicted_probabilities = cnn.predict(images)
predicted_classes = np.argmax(predicted_probabilities, axis=1)

# Define class names manually or from the validation set
class_names = ["Angioectasia", "Bleeding", "Erosion", "Erythema", "Foreign Body", 
               "Lymphangiectasia", "Normal", "Polyp", "Ulcer", "Worms"]

# Prepare data for CSV
data = {
    "image_path": image_paths,
}

# Add predicted probabilities for each class
for i, class_name in enumerate(class_names):
    data[class_name] = [prob[i] for prob in predicted_probabilities]

# Add predicted class names
data["predicted_class"] = [class_names[i] for i in predicted_classes]

# Convert to DataFrame and save as CSV
df = pd.DataFrame(data)
output_csv = 'cnn10team_name_predictio11ns.csv'
df.to_csv(output_csv, index=False)
print(f"CSV file saved as: {output_csv}")


138/138 ━━━━━━━━━━━━━━━━━━━━ 14s 102ms/step
CSV file saved as: team_name_predictio11ns.csv
